In [1]:
import numpy as np
np.random.seed(3)
from pybedtools import BedTool
import pybedtools
import pandas as pd
import seaborn as sns

In [2]:
def read_data(bed_file,fasta_file):
    #apply bedtools to read fasta files '/home/h5li/methylation_DMR/data/DMR_coordinates_extended_b500.bed'
    a = pybedtools.example_bedtool( bed_file )
    # '/home/h5li/methylation_DMR/data/mm10.fasta'
    fasta = pybedtools.example_filename( fasta_file )
    a = a.sequence(fi=fasta)
    seq = open(a.seqfn).read()
    #read and extract DNA sequences 
    DNA_seq_list = seq.split('\n')
    DNA_seq_list.pop()
    DNA_seq = []
    m = 10000
    n = 0
    for index in range(len(DNA_seq_list)//2):
        DNA_seq.append(DNA_seq_list[index*2 + 1].upper())
        if len(DNA_seq_list[index*2 + 1]) < m:
            m = len(DNA_seq_list[index*2 + 1])
        if len(DNA_seq_list[index*2 + 1]) > n:
            n = len(DNA_seq_list[index*2 + 1])
    print('The shortest length of DNA sequence is {0}bp'.format(m))
    print('The longest length of DNA sequence is {0}bp'.format(n))
    print('Total Number of input sequence is {0}'.format(len(DNA_seq)))
    return DNA_seq,n,m

In [3]:
bed_file_path = '/home/h5li/methylation_DMR/data/DMR_coordinates_extended_b500.bed'
fasta_file_path = '/home/h5li/methylation_DMR/data/mm10.fasta'
DNA_seq,long_length,short_length = read_data(bed_file_path, fasta_file_path)   

The shortest length of DNA sequence is 1407bp
The longest length of DNA sequence is 5829bp
Total Number of input sequence is 58959


In [4]:
def chop_Data(targetLength,dnaSeqList):
    #chop DNA sequences to have same length
    Uni_DNA = []
    for s in dnaSeqList:
        if len(s) < targetLength:
            print('Exceptions!')
        diff = len(s) - targetLength
        if diff % 2 == 0:
            side = diff // 2
            Uni_DNA.append(s[side:-side])
        else:
            right = diff // 2
            left = diff// 2 + 1
            Uni_DNA.append(s[left:-right])
    return Uni_DNA

In [5]:
target_length = 600
DNA_seq = chop_Data(target_length,DNA_seq)

In [6]:
Kmers = pd.read_csv('../../data/Kmers_K6_sequences.csv',header = None)
Kmers = Kmers.as_matrix().reshape(1,-1)[0]

In [7]:
result_list = [[] for x in range(len(DNA_seq))]

In [8]:
for i,d in enumerate(DNA_seq):
    if i % 1000 == 0:
        print(i)
    for c in Kmers:
        result_list[i].append(d.count(c))

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000


In [9]:
np.array(result_list).shape

(58959, 2080)

In [10]:
df = pd.DataFrame(data = np.array(result_list),columns = list(Kmers),dtype = 'int')

In [11]:
df.head()

,CCCCCC,CCCCCT,CCCCCA,CCCCCG,CCCCTC,CCCCTT,CCCCTA,CCCCTG,CCCCAC,CCCCAT,...,GGCCCC,GGCTCC,GGCACC,GGCGCC,GGTCCC,GGTTCC,GGTACC,GGACCC,GGATCC,GGGCCC
0,0,0,0,0,0,1,0,1,0,0,...,0,0,0,0,1,0,1,0,0,0
1,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,1,...,0,1,0,0,0,2,0,0,0,0


In [12]:
df.to_csv('Kmers6_counts_600bp.csv',index = False)

In [13]:
df.shape

(58959, 2080)